# 测试，爬取固定网页下载链接等信息

In [1]:
import requests
from bs4 import BeautifulSoup
import re

T=[]           # 保存全部下载信息

# Send an HTTP request to the website
url = 'https://civitai.com/models/42374'              # 链接
headers = {
"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36",
"cookie": "__Secure-next-auth.callback-url=https%3A%2F%2Fcivitai.com; __Host-next-auth.csrf-token=19a63269ab01f9b2fdf113515910e49f5848de590e88f643a5f218dfa47b79df%7C55738fc76b994da04af00dc500f69b3b669265a3a5310ed95f2d7223c24c3aa5; __stripe_mid=792923b6-7bff-4c39-a786-5560731c7604e2d71b; f_sort=Newest; __stripe_sid=6781e6d8-bf7f-4d62-b302-4fa4b864d090ef5702",
}              # request浏览器信息   
 
response = requests.get(url,headers=headers)     # request

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')    

# download link
links =[]
# download links
for link in soup.find_all('a'):
    href = link.get('href')
    if href:
        if href[1:4]=="api":
            if href not in links:
                links.append(href)
# string webpage
soup1=str(soup)     

# links有信息则进行下一步操作，爬取下一个网页
if links:
    
    # downloadlink
    downloadlink="https://civitai.com"+links[0]
    
    # name
    name=re.findall(r"mantine-Text-root mantine-Title-root mantine-g96yxx\">(.+?)</h1>",soup1)

    # uploaded date
    uploaded=re.findall(r"Uploaded</div></td><td class=\"mantine-1avyp1d\">(.+?)</td>",soup1)
    
    # label
    label=re.findall(r"format=(.+?)\" type=",soup1)

    # comments
    comments=[]
    for link in soup.find_all('div'):
        for linkk in soup.find_all('div'):
            link1=str(linkk)
            comment=re.findall(r"<div><p>(.+?)</p></div></div></div>",link1)
            commentt=re.findall(r"<div><p><strong>(.+?)</strong></p><p></p>",link1)
            if comment:
                if comment[0] not in comments:
                    comments.append(comment[0])
            else:
                if commentt:
                    if commentt[0] not in comments:
                        comments.append(commentt[0])
                else:
                    if comments:
                        t=0
                    else:
                        comments.append(" ")
            
print(comments)
# 全部内容保存成字典
tex={"downloadlink":downloadlink,
    "name":name[0],
    "uploaded date":uploaded[0],
    "comments":comments[0]}

#print(tex)


['IF YOU ARE NEW TO STABLE DIFFUSION OR YOU HAVE PROBLEMS WITH MY MODELS GO HERE: </strong><a href="https://inzaniak.github.io/guide" rel="ugc" target="_blank"><strong>Guide</strong></a></p><p><strong>RIGHT NOW IMAGES METADATA ARE MISSING AS I\'M USING COMFYUI TO SIMPLIFY THE IMAGE GENERATION. I\'M TRYING TO FIND A WAY TO FIX THIS, IF SOMEONE KNOWS HOW TO DO IT, I\'D LIKE TO KNOW.']


IndexError: list index out of range

In [4]:
downloadlink

'https://civitai.com/api/download/models/47051'

In [22]:
from urllib.parse import unquote

def get_file_name(url, headers):
    filename = ''
    if 'Content-Disposition' in headers and headers['Content-Disposition']:
        disposition_split = headers['Content-Disposition'].split(';')
        if len(disposition_split) > 1:
            if disposition_split[1].strip().lower().startswith('filename='):
                file_name = disposition_split[1].split('=')
                if len(file_name) > 1:
                    filename = unquote(file_name[1])
    if not filename and os.path.basename(url):
        filename = os.path.basename(url).split("?")[0]
    if not filename:
        return time.time()
    return filename

url='https://civitai.com/api/download/models/47051'
get_file = requests.get(url=url, headers=headers, stream=True, allow_redirects=True, timeout=10)

file_name = get_file_name(url, get_file.headers)[1:-1]


In [25]:
file_name

'badpaint.safetensors'

# 爬取全网站的全部链接

In [5]:
from multiprocessing.pool import ThreadPool
from bs4 import BeautifulSoup
import requests
import urllib
import time
import re
import os

T=[]                   # 保存全部下载信息
downloadlinks = []     # 保存全部下载链接
names = []             # 保存全部名称
uploadeds = []         # 保存全部上传时间
commentss = []         # 保存全部评论
imagelinks = []        # 保存全部图片链接
labels = []            # 保存全部标签
count=0

for i in range(355,399):
    # Send an HTTP request to the website
    url = 'https://civitai.com/models/42'+str(i)
    headers = {
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36",
        "cookie": "__Secure-next-auth.callback-url=https%3A%2F%2Fcivitai.com; __Host-next-auth.csrf-token=19a63269ab01f9b2fdf113515910e49f5848de590e88f643a5f218dfa47b79df%7C55738fc76b994da04af00dc500f69b3b669265a3a5310ed95f2d7223c24c3aa5; __stripe_mid=792923b6-7bff-4c39-a786-5560731c7604e2d71b; f_sort=Newest; __stripe_sid=6781e6d8-bf7f-4d62-b302-4fa4b864d090ef5702",
    }
    
    response = requests.get(url,headers=headers)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # download link
    links =[]
    # download links
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            if href[1:4]=="api":
                if href not in links:
                    links.append(href)
    # string webpage
    soup1=str(soup)
    count = count+1       # 连续空白网页数量
    # links有信息则进行下一步操作，爬取下一个网页
    if links:
        print("正在爬取网页："+str(i))
        count = 0        # 遇到非空白网页置计数器为0
        # downloadlink
        downloadlink="https://civitai.com"+links[0]
        downloadlinks.append(downloadlink)
        
        # name
        name=re.findall(r"mantine-Text-root mantine-Title-root mantine-g96yxx\">(.+?)</h1>",soup1)
        names.append(name[0])
        
        # uploaded date
        uploaded=re.findall(r"Uploaded</div></td><td class=\"mantine-1avyp1d\">(.+?)</td>",soup1)
        uploadeds.append(uploaded[0])
        
        # images links
        imagelink = []
        for link in soup.find_all('div'):
            #print(link)
            link1=str(link)
            imagesrc=re.findall(r"src=\"(.+?).jpeg",link1)
            for i in range(0,len(imagesrc)):
                if imagesrc[i] not in imagelink:
                    imagelink.append(imagesrc[i]+".jpeg")
        imagelinks.append(imagelink)
        
        # label
        label=re.findall(r"format=(.+?)\" type=",soup1)
        labels.append(label[0])
        
        # comments
        comments=[]
        for link in soup.find_all('div'):
            for linkk in soup.find_all('div'):
                link1=str(linkk)
                comment=re.findall(r"<div><p>(.+?)</p></div></div></div>",link1)
                commentt=re.findall(r"<div><p><strong>(.+?)</strong></p><p></p>",link1)
                if comment:
                    comm=comment[0].replace("</p><p><br/>", "/")
                    comm=comm.replace("</p><p>", "/")
                    if comm not in comments:
                        comments.append(comm)
                if commentt:
                    comm=commentt[0].replace("<strong>", "/")                    
                    if comm not in comments:
                        comments.append(comm)
            else:
                comments.append(' ')
        commentst = []
        for k in comments:
            if k!=' ':
                # print(k)
                commentst.append(k)
        if commentst: 
            commentss.append(commentst)
        else:
            commentss.append(' ')
            
        # 全部内容保存成字典
        tex={"downloadlink":downloadlinks[-1],
             "name":names[-1],
             "uploaded date":uploadeds[-1],
             "images links":imagelinks[-1],
             "labels": labels[-1],
             "comments":commentss[-1]}
        
        # 全部信息
        T.append(tex)

#print(T)


正在爬取网页：355
正在爬取网页：356
正在爬取网页：357
正在爬取网页：358
正在爬取网页：359
正在爬取网页：360
正在爬取网页：361
正在爬取网页：362
正在爬取网页：363
正在爬取网页：364
正在爬取网页：365
正在爬取网页：366
正在爬取网页：367
正在爬取网页：368
正在爬取网页：369
正在爬取网页：370
正在爬取网页：371
正在爬取网页：372
正在爬取网页：373
正在爬取网页：374
正在爬取网页：375
正在爬取网页：376
正在爬取网页：377
正在爬取网页：378
正在爬取网页：379
正在爬取网页：380
正在爬取网页：381
正在爬取网页：382
正在爬取网页：383
正在爬取网页：384
正在爬取网页：385
正在爬取网页：386
正在爬取网页：387
正在爬取网页：388
正在爬取网页：389
正在爬取网页：390
正在爬取网页：391
正在爬取网页：392
正在爬取网页：393
正在爬取网页：394
正在爬取网页：395
正在爬取网页：396
正在爬取网页：397
正在爬取网页：398


# 根据主网页创建文件夹

In [6]:
from urllib.parse import unquote

def get_file_name(url, headers):
    filename = ''
    if 'Content-Disposition' in headers and headers['Content-Disposition']:
        disposition_split = headers['Content-Disposition'].split(';')
        if len(disposition_split) > 1:
            if disposition_split[1].strip().lower().startswith('filename='):
                file_name = disposition_split[1].split('=')
                if len(file_name) > 1:
                    filename = unquote(file_name[1])
    if not filename and os.path.basename(url):
        filename = os.path.basename(url).split("?")[0]
    if not filename:
        return time.time()
    return filename

# 创建并进入文件夹
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
        os.chdir(path)
    else:
        os.chdir(path)

# 保存评论为txt
def txt(name,text):              #定义函数名
    folder = os.path.exists(name+'.txt')
    if not folder:
        print("正在保存"+name+"的评论")
        filename = name + '.txt'    #在当前py文件所在路径下的new文件中创建txt
        file = open(filename,'w')
        file.write(text)        #写入内容信息
        file.close()
    else:
        print("评论已保存！")

folders = "/Users/py/Documents/AEssay/20230417爬虫"      # 根目录############  需要修改

os.chdir(folders)           # 进入folders文件夹

rootpath = "Civital"
folder = os.path.exists(folders+'/'+rootpath)
if not folder:
    os.makedirs("Civital")
    os.chdir(folders+'/'+rootpath)
else:
    os.chdir(folders+'/'+rootpath)

for i in range(0,len(names)):
    os.chdir(folders+'/'+rootpath)
    file = names[i].split('/')[0]
    mkdir(file)                                    #调用函数
    txt(names[i].split('/')[0],str(commentss[i]))       #创建名称为name[i]的txt文件，内容为comments

# 查看当前工作目录
retval = os.getcwd()


正在保存Hoshino Ai的评论
正在保存Ai Hoshino [Oshi no Ko]的评论
正在保存BadPaint - Style的评论
正在保存Punishing Gray Raven Luna Laurel Lora的评论
正在保存YabaL_Mix V2的评论
正在保存Aquariumgirl#的评论
正在保存Yu-Gi-Oh! ティアラメンツ・シェイレーン|Tearalaments Kitkalos|珠泪哀歌族·水仙女人鱼的评论
正在保存3D CG Style| Realistic的评论
正在保存Yakui的评论
正在保存Corto Heilani (Droners)的评论
正在保存Ib イヴ - Ib的评论


# 下载各网页的链接、图、评论

In [7]:

# 下载文件
def url_response(url):
    time.sleep(3)           # 防封，暂停3秒
    path, url, t = url
    r = requests.get(url, stream=True)
    path1 = retval+"/"+path
    paths = retval+"/"+path+"/"+t
    if os.path.isfile(paths):
        print("文件已下载！保存在："+path1)
    else:
        print("正在下载"+path1)
        with open(paths, 'wb') as f:
            for ch in r:
                f.write(ch)
os.chdir(folders+"/Civital")

urls = []
for i in range(0,len(downloadlinks)):
    nm = names[i].split('/')[0]
    t = downloadlinks[i].split('/')[-1]+"."+labels[i]
    if i>0:
        if downloadlinks[i] not in urls[-1][1]:
            urls.append((nm,downloadlinks[i],t))
    else:
        urls.append((nm,downloadlinks[i],t))
    if imagelinks[i]:
        if len(imagelinks[i][0])>1:
            for j in range(0,len(imagelinks[i])):
                nm = names[i].split('/')[0]
                t = imagelinks[i][j].split('/')[-1].split('.')[0]+'.jpeg'
                if i>0:
                    if imagelinks[i][j] not in urls[-1][1]:
                        urls.append((nm,imagelinks[i][j],t))
                        #print(urls[-1])
                    else:
                        urls.append((nm,imagelinks[i][j],t))
        else:
            nm = names[i].split('/')[0]
            t = imagelinks[i].split('/')[-1].split('.')[0]+'.jpeg'
            if i>0:
                if imagelinks[i][j] not in urls[-1][1]:
                    urls.append((nm,imagelinks[i],t))
                    #print(urls[-1])
                else:
                    urls.append((nm,imagelinks[i],t))

start = time.time()
ThreadPool(9).imap_unordered(url_response, urls)        # 并行下载

retval = os.getcwd()





文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]

文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]

文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Hoshino Ai
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Ai Hoshino [Oshi no Ko]
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civit

文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Punishing Gray Raven Luna Laurel Lora
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Punishing Gray Raven Luna Laurel Lora
文件已下载！保存在：/Users/py/Documents/AEssay/20230417爬虫/Civital/Punishing Gray Raven Luna Laurel Lora
